# Data Exploration
<hr style="clear:both">

In this notebook, we present our journey to merge and prepare for analysis the three datasets we used for our project:
 * [CMU Movie Summary Corpus](http://www.cs.cmu.edu/~ark/personas/)
 * [Full TMDB Movies Dataset 2024](https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/data)
 * [MovieLens Tag Genome Dataset 2021](https://grouplens.org/datasets/movielens/) 

More precisely, we will explore how we incorporated the sentiment analysis of the movie summaries into our dataset (for more details, refer to the `sentiment_analysis.ipynb` notebook) and how we adjusted the box office revenue to account for inflation.

**Project Mentor:** [Shuo Wen](http://personnes.epfl.ch/shuo.wen) ([Email](shuo.wen@epfl.ch)),
**Authors:** Mahmoud Dokmak, Matthieu Borello, Léo Brunneau, Loïc Domingos, Bastien Armstrong

<hr style="clear:both">

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import cpi
import tqdm
from tqdm import tqdm
from src.models.Impact import impact_genre
import matplotlib.cm as cm
from src.models.PolynomialRegression import polynomial_regression
from src.models.ARIMA import arima
from src.utils.print_prediction import plot_results
import warnings
warnings.filterwarnings('ignore')
from typing import final
from src.utils.results_utils import *
from matplotlib import pyplot as plt
from src.utils.results_utils import *

## Loading the "_CMU Movie Summary Corpus_" dataset

In [3]:
# Giving an appropriate name to the columns
movie_headers = ["Wikipedia movie ID", "Freebase movie ID", "Movie name", "Movie release date", "Movie box office revenue", "Movie runtime", "Movie languages", "Movie countries", "Movie genres"]
movie_headers = [header.lower().replace(' ', '_') for header in movie_headers]

In [4]:
# Loading the CMU Movie Dataset and showing the first 5 rows
movies_df = pd.read_csv("./data/MovieSummaries/movie.metadata.tsv", sep="\t", header=None, names=movie_headers)
print_df_size("MoviesSummaries", movies_df)
movies_df.head()

MoviesSummaries - shape: (81741, 9) --> 81741 rows and 9 columns


,wikipedia_movie_id,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,movie_runtime,movie_languages,movie_countries,movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


## Loading the "_TMDB_" dataset

In [6]:
# Loading the TMDB Movie Dataset and showing the first 5 rows
tmdb_movies_df = pd.read_csv("./data/TMDBMovies/TMDB_movie_dataset_v11.csv")
print_df_size("TMDB", tmdb_movies_df)
tmdb_movies_df.head()

TMDB - shape: (1149258, 24) --> 1149258 rows and 24 columns


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


## Merging the "_CMU Movie Summary Corpus_" with the "_TMDB_" datasets

To be able to merge our datasets, we needed to create a converter dataset to map `IMDB Id` to `Freebase Id`. To achieve this, we used the [Wikipedia Query Service](https://query.wikidata.org/#SELECT%20%3Fimdb_id%20%3Ffreebase_id%20WHERE%20%7B%0A%20%20%3Fmovie%20wdt%3AP345%20%3Fimdb_id%20.%20%20%20%23%20IMDb%20ID%0A%20%20%3Fmovie%20wdt%3AP646%20%3Ffreebase_id%20.%20%23%20Freebase%20ID%0A%7D) with the following query:

```sparql
SELECT ?imdb_id ?freebase_id WHERE {
  ?movie wdt:P345 ?imdb_id .   # IMDb ID
  ?movie wdt:P646 ?freebase_id . # Freebase ID
}
```

In [7]:
# Loading the Converter file and merging the CMU Movie Summary Corpus with the TMDB datasets on the freebase ids
query_df = pd.read_csv("./data/Converter/query.csv", sep=',', header=0)
tmdb_with_ids_df = tmdb_movies_df.merge(query_df, how='inner', on='imdb_id')
full_merged = pd.merge(movies_df, tmdb_with_ids_df, left_on='freebase_movie_id', right_on='freebase_id', how='inner')
print_df_size("CMU & TMDB", full_merged)
full_merged.head()

CMU & TMDB - shape: (68468, 34) --> 68468 rows and 34 columns


,wikipedia_movie_id,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,movie_runtime,movie_languages,movie_countries,movie_genres,id,...,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,freebase_id
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",10016,...,"In 2176, a Martian police unit is sent to pick...",14.189,/i2zztssCIbahGES1fdfWFmDXian.jpg,Terror is the same on any planet.,"Action, Horror, Science Fiction","Animationwerks, Screen Gems, Storm King Produc...",United States of America,English,"future, planet mars, anti hero, possession, ho...",/m/03vyhn
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",784579,...,Dramatization of the story behind the murder o...,0.600,NaN,NaN,"Drama, Crime",NaN,United States of America,English,"colorado, jonbenet",/m/08yl5d
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",396302,...,A stolen bicycle case ends with drunken detect...,0.898,/rtFEOQQ0LVDv7y2bCgVVg3wWpXK.jpg,A film about murder and love,"Mystery, Crime, Drama","Filmeffekt AS, Norsk Film",Norway,Norwegian,"bicycle, private detective",/m/0crgdbh
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",33592,...,"In a wealthy and isolated desert community, a ...",8.297,/aLFoGmQpknOvcyx4imCP7Fuvoip.jpg,No woman is safe… while he is loose!,"Horror, Thriller",Mrs. White's Productions,United Kingdom,English,"based on novel or book, gas station, psychopat...",/m/0285_cd
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",11192,...,"Eva, an upper-class housewife, frustratedly le...",2.801,/iKeCVR0x8vnRXLtjwjZ7EuVFlo7.jpg,NaN,Drama,Dieter Geissler Filmproduktion,Germany,German,"jealousy, eroticism, gigolo, longing, dominatr...",/m/01mrr1


In [8]:
# Keeping only the useful columns
full_reduced = merge_cmu_tmdb_clean_columns(full_merged)
full_reduced.head()

,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,movie_countries,movie_genres,vote_average,vote_count,release_date,revenue,popularity,genres,production_countries
0,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",5.127,977,2001-08-24,14010832,14.189,"Action, Horror, Science Fiction",United States of America
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,"{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",0.000,0,2000-02-16,0,0.600,"Drama, Crime",United States of America
2,/m/0crgdbh,Brun bitter,1988,NaN,"{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",0.000,0,1988-11-17,0,0.898,"Mystery, Crime, Drama",Norway
3,/m/0285_cd,White Of The Eye,1987,NaN,"{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",5.742,64,1987-06-19,0,8.297,"Horror, Thriller",United Kingdom
4,/m/01mrr1,A Woman in Flames,1983,NaN,"{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",5.300,13,1983-05-11,0,2.801,Drama,Germany


In [9]:
# Cleaning the columns that contain dictionaries
full_reduced = merge_cmu_tmdb_remove_dictionaries(full_reduced)
full_reduced.head()

,freebase_movie_id,movie_name,movie_release_date,movie_box_office_revenue,vote_average,vote_count,release_date,revenue,popularity,movie_countries_final,movie_genres_final
0,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,5.127,977,2001-08-24,14010832,14.189,United States of America,"Thriller, Science Fiction, Horror, Adventure, ..."
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,0.000,0,2000-02-16,0,0.600,United States of America,"Mystery, Biographical film, Drama, Crime Drama"
2,/m/0crgdbh,Brun bitter,1988,NaN,0.000,0,1988-11-17,0,0.898,Norway,"Crime Fiction, Drama"
3,/m/0285_cd,White Of The Eye,1987,NaN,5.742,64,1987-06-19,0,8.297,United Kingdom,"Thriller, Erotic thriller, Psychological thriller"
4,/m/01mrr1,A Woman in Flames,1983,NaN,5.300,13,1983-05-11,0,2.801,Germany,Drama


In [10]:
# Combine the columns that contains the same information
full_reduced = merge_cmu_tmdb_combine_columns(full_reduced)
full_reduced.head()

,freebase_movie_id,movie_name,vote_average,vote_count,popularity,movie_countries_final,movie_genres_final,combined_release_date,Box_Office
0,/m/03vyhn,Ghosts of Mars,5.127,977,14.189,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",2001-08-24,14010832.0
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,0.000,0,0.600,United States of America,"Mystery, Biographical film, Drama, Crime Drama",2000-02-16,NaN
2,/m/0crgdbh,Brun bitter,0.000,0,0.898,Norway,"Crime Fiction, Drama",1988-11-17,NaN
3,/m/0285_cd,White Of The Eye,5.742,64,8.297,United Kingdom,"Thriller, Erotic thriller, Psychological thriller",1987-06-19,NaN
4,/m/01mrr1,A Woman in Flames,5.300,13,2.801,Germany,Drama,1983-05-11,NaN


In [11]:
# Drop the rows that have either empty release dates or genres or countries because these are the main features we will use
full_reduced = full_reduced.dropna(subset=['combined_release_date', 'movie_genres_final', 'movie_countries_final', 'popularity'])
print_df_size("full_reduced", full_reduced)
full_reduced.head()

full_reduced - shape: (67174, 9) --> 67174 rows and 9 columns


,freebase_movie_id,movie_name,vote_average,vote_count,popularity,movie_countries_final,movie_genres_final,combined_release_date,Box_Office
0,/m/03vyhn,Ghosts of Mars,5.127,977,14.189,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",2001-08-24,14010832.0
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,0.000,0,0.600,United States of America,"Mystery, Biographical film, Drama, Crime Drama",2000-02-16,NaN
2,/m/0crgdbh,Brun bitter,0.000,0,0.898,Norway,"Crime Fiction, Drama",1988-11-17,NaN
3,/m/0285_cd,White Of The Eye,5.742,64,8.297,United Kingdom,"Thriller, Erotic thriller, Psychological thriller",1987-06-19,NaN
4,/m/01mrr1,A Woman in Flames,5.300,13,2.801,Germany,Drama,1983-05-11,NaN


In [12]:
#save this dataframe in a pickle file for faster loading
full_reduced.to_pickle("./data/CMU_and_TDB.pkl")

## Loading the "_MovieLens_" dataset

In [13]:
# Load the MovieLens dataset
added_movies = pd.read_json('./data/MovieLens/raw/metadata_updated.json', lines=True)
added_movies["imdbId"] = added_movies["imdbId"].astype(str)
print_df_size("MovieLens", added_movies)
added_movies.head()

MovieLens - shape: (84661, 6) --> 84661 rows and 6 columns


,title,directedBy,starring,avgRating,imdbId,item_id
0,Toy Story (1995),John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146,114709,1
1,Jumanji (1995),Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605,113497,2
2,Grumpier Old Men (1995),Howard Deutch,"Jack Lemmon, Walter Matthau, Ann-Margret , Sop...",3.17146,113228,3
3,Waiting to Exhale (1995),Forest Whitaker,"Angela Bassett, Loretta Devine, Whitney Housto...",2.86824,114885,4
4,Father of the Bride Part II (1995),Charles Shyer,"Steve Martin, Martin Short, Diane Keaton, Kimb...",3.07620,113041,5


## Merging the "_CMU Movie Summary Corpus_", "_TMDB_" and "_MovieLens_" datasets

In [14]:
# Loading the Converter file and merging the CMU Movie Summary Corpus with the TMDB datasets on the freebase ids
converter = pd.read_csv('./data/Converter/query.csv')
converter["imdb_id"] = converter["imdb_id"].str[3:]
merged_data = pd.merge(full_reduced, converter, left_on="freebase_movie_id", right_on="freebase_id", how='inner')
merged_data = merged_data.drop_duplicates(subset='freebase_movie_id', keep='first')

In [15]:
# Merge the MovieLens data with the two already merged datasets
final_data = pd.merge(merged_data, added_movies, left_on='imdb_id', right_on="imdbId", how='left')
columns_to_drop = ['title','directedBy', 'starring', 'imdbId', 'freebase_id']
filtered_final_data = final_data.drop(columns=columns_to_drop)
print_df_size("filtered final data", filtered_final_data)
filtered_final_data.head()

filtered final data - shape: (67119, 12) --> 67119 rows and 12 columns


,freebase_movie_id,movie_name,vote_average,vote_count,popularity,movie_countries_final,movie_genres_final,combined_release_date,Box_Office,imdb_id,avgRating,item_id
0,/m/03vyhn,Ghosts of Mars,5.127,977,14.189,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",2001-08-24,14010832.0,228333,2.35723,4735.0
1,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,0.000,0,0.600,United States of America,"Mystery, Biographical film, Drama, Crime Drama",2000-02-16,NaN,245916,NaN,NaN
2,/m/0crgdbh,Brun bitter,0.000,0,0.898,Norway,"Crime Fiction, Drama",1988-11-17,NaN,094806,NaN,NaN
3,/m/0285_cd,White Of The Eye,5.742,64,8.297,United Kingdom,"Thriller, Erotic thriller, Psychological thriller",1987-06-19,NaN,094320,NaN,NaN
4,/m/01mrr1,A Woman in Flames,5.300,13,2.801,Germany,Drama,1983-05-11,NaN,083949,NaN,NaN


In [16]:
# Save the final data in a pickle file for faster loading
filtered_final_data.to_pickle("./data/Three_Datasets_Combined.pkl")

## Loading the "_MovieLens Reviews_" dataset
This is part of our MovieLens dataset, it contains the reviews of the movies.

In [17]:
# Load the reviews dataset, as it takes a lot of time to load we directly save it in a pickle file
reviews_df = pd.read_json('./data/MovieLens/raw/reviews.json', lines=True)
reviews_df.to_pickle('./data/reviews.pkl')

## Preparing the reviews dataset for the sentiment analysis

In [19]:
# For faster loading we will load the reviews from the pickle file
reviews_df = pd.read_pickle('./data/reviews.pkl')
print_df_size("Number of reviews", reviews_df)
print("The number of unique movies in the reviews dataset is: ", reviews_df['item_id'].nunique())
reviews_df.head()

Number of reviews - shape: (2624608, 2) --> 2624608 rows and 2 columns
The number of unique movies in the reviews dataset is:  52081


,item_id,txt
0,172063,"one-shot record of a belly dancer; ""Carmencita..."
1,95541,Banging Away..; Thomas Edison and William K.L....
2,7065,unbelievable; I cannot understand how anyone c...
3,3739,I'm still starry-eyed from it; I saw this last...
4,1562,Failed on every Front; Joel Schumacher who did...


In [20]:
# We are only going to apply sentiment analysis on the reviews that are available in our merged dataset
# This particular step is to keep track of the review number/order
reviews_df = reviews_df.reset_index()
merged_reviews = pd.merge(reviews_df, filtered_final_data, left_on='item_id', right_on='item_id', how='inner')
merged_reviews = merged_reviews.sort_values(by='item_id')
merged_reviews = merged_reviews.rename(columns={'txt': 'review'})
print_df_size("merged reviews", merged_reviews)
merged_reviews.head()

merged reviews - shape: (1293847, 14) --> 1293847 rows and 14 columns


,index,item_id,review,freebase_movie_id,movie_name,vote_average,vote_count,popularity,movie_countries_final,movie_genres_final,combined_release_date,Box_Office,imdb_id,avgRating
260872,412847,1,Fantastic Revolutionary Computer Animation Cla...,/m/0dyb1,Toy Story,7.971,17152,78.404,United States of America,"Buddy film, Adventure, Children's/Family, Comp...",1995-10-30,361958736.0,114709,3.89146
828262,2044033,1,The best film for small children....; My toddl...,/m/0dyb1,Toy Story,7.971,17152,78.404,United States of America,"Buddy film, Adventure, Children's/Family, Comp...",1995-10-30,361958736.0,114709,3.89146
463118,757613,1,Toy Story was like the Snow White of 1995; I r...,/m/0dyb1,Toy Story,7.971,17152,78.404,United States of America,"Buddy film, Adventure, Children's/Family, Comp...",1995-10-30,361958736.0,114709,3.89146
642563,1325676,1,Timeless Animation; Toy Story came out right i...,/m/0dyb1,Toy Story,7.971,17152,78.404,United States of America,"Buddy film, Adventure, Children's/Family, Comp...",1995-10-30,361958736.0,114709,3.89146
409172,658384,1,A timeless classic.; Toy Story is truly a grou...,/m/0dyb1,Toy Story,7.971,17152,78.404,United States of America,"Buddy film, Adventure, Children's/Family, Comp...",1995-10-30,361958736.0,114709,3.89146


In [21]:
# Final dataset for sentiment analysis
merged_reviews.to_pickle('./data/merged_data_with_reviews.pkl')

## Loading the sentiment analysis output
For more information on sentiment analysis, see the notebook sentiment_analysis.ipynb

In [22]:
# Loading the sentiment analysis dataset saved in its notebook while performing the sentiment analysis
df_sa = pd.read_csv('./data/sentiment_analysis.csv')
df_sa.head()

,Unnamed: 0,sentiment_analysis,item_id,index
0,828299,"{'label': 'NEGATIVE', 'score': 0.9858629107475...",1,2044046
1,828302,"{'label': 'POSITIVE', 'score': 0.9998522996902...",1,2044054
2,828305,"{'label': 'POSITIVE', 'score': 0.9990456700325...",1,2044063
3,828306,"{'label': 'POSITIVE', 'score': 0.9998844861984...",1,2044069
4,828308,"{'label': 'POSITIVE', 'score': 0.9972469806671...",1,2044077


In [23]:
# Split the sentiment_analysis column into two columns, one for the label and one for the score
df_sa['sentiment_analysis'] = df_sa['sentiment_analysis'].apply(eval)
df_sa[['label', 'score']] = df_sa['sentiment_analysis'].apply(pd.Series)
# Remove unnecessary columns
df_sa.drop(columns=['sentiment_analysis', "Unnamed: 0", "index"], inplace=True)
# Replace the labels with 0 and 1
df_sa['label'] = df_sa['label'].replace({'NEGATIVE': 0, 'POSITIVE': 1})
df_sa.head()

,item_id,label,score
0,1,0,0.985863
1,1,1,0.999852
2,1,1,0.999046
3,1,1,0.999884
4,1,1,0.997247


In [24]:
# getting the mean per item id and apply penalty for items that have low number of reviews
mean_sa = df_sa.groupby('item_id')['label'].mean().reset_index()
mean_sa['count'] = df_sa.groupby('item_id').size().reset_index(name='count')['count']
mean_sa['penalty'] = 1 - 0.5**mean_sa['count']
mean_sa['sa_score']=mean_sa['label']*mean_sa['penalty']
mean_sa.drop(columns=['label', 'count', 'penalty'], inplace=True)
mean_sa.head()

,item_id,sa_score
0,1,0.912114
1,2,0.775148
2,3,0.716667
3,4,0.632653
4,5,0.666667


## Adding the sentiment analysis score to the dataset

Now we add the sentiment analysis score to our dataset.

In [25]:
df_three_datasets_combined = pd.read_pickle("./data/Three_Datasets_Combined.pkl")
df_final = df_three_datasets_combined.merge(mean_sa, on='item_id', how='left')
# Specify the desired order of columns
desired_order = ['freebase_movie_id', 'imdb_id', 'item_id', 'movie_name', 'combined_release_date', 'movie_countries_final', 'movie_genres_final', 'Box_Office', 'sa_score', 'avgRating', 'vote_average', 'vote_count', 'popularity']  # Replace with your column names
df_final = df_final.reindex(columns=desired_order)
df_final.head()

,freebase_movie_id,imdb_id,item_id,movie_name,combined_release_date,movie_countries_final,movie_genres_final,Box_Office,sa_score,avgRating,vote_average,vote_count,popularity
0,/m/03vyhn,228333,4735.0,Ghosts of Mars,2001-08-24,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",14010832.0,0.351293,2.35723,5.127,977,14.189
1,/m/08yl5d,245916,NaN,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,United States of America,"Mystery, Biographical film, Drama, Crime Drama",NaN,NaN,NaN,0.000,0,0.600
2,/m/0crgdbh,094806,NaN,Brun bitter,1988-11-17,Norway,"Crime Fiction, Drama",NaN,NaN,NaN,0.000,0,0.898
3,/m/0285_cd,094320,NaN,White Of The Eye,1987-06-19,United Kingdom,"Thriller, Erotic thriller, Psychological thriller",NaN,NaN,NaN,5.742,64,8.297
4,/m/01mrr1,083949,NaN,A Woman in Flames,1983-05-11,Germany,Drama,NaN,NaN,NaN,5.300,13,2.801


Saving the three datasets merged and with the sentiment analysis score.

In [26]:
# Save the dataset with sentiment analysis score in a pickle file
df_final.to_pickle("./data/OurFinalDataset.pkl")

# Box Office Revenue with Inflation

Now we still need to take into account the inflation for the box office revenue.

In [5]:
# We load the final dataset.
final_dataset = pd.read_pickle('./data/OurFinalDataset.pkl')
final_dataset.replace(0, np.nan, inplace=True)
final_dataset.dropna(subset=['popularity'], inplace=True)
final_dataset.head()

,freebase_movie_id,imdb_id,item_id,movie_name,combined_release_date,movie_countries_final,movie_genres_final,Box_Office,sa_score,avgRating,vote_average,vote_count,popularity
0,/m/03vyhn,228333,4735.0,Ghosts of Mars,2001-08-24,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",14010832.0,0.351293,2.35723,5.127,977.0,14.189
1,/m/08yl5d,245916,NaN,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,United States of America,"Mystery, Biographical film, Drama, Crime Drama",NaN,NaN,NaN,NaN,NaN,0.600
2,/m/0crgdbh,094806,NaN,Brun bitter,1988-11-17,Norway,"Crime Fiction, Drama",NaN,NaN,NaN,NaN,NaN,0.898
3,/m/0285_cd,094320,NaN,White Of The Eye,1987-06-19,United Kingdom,"Thriller, Erotic thriller, Psychological thriller",NaN,NaN,NaN,5.742,64.0,8.297
4,/m/01mrr1,083949,NaN,A Woman in Flames,1983-05-11,Germany,Drama,NaN,NaN,NaN,5.300,13.0,2.801


In [6]:
# Checking for missing values
final_dataset.isna().sum()

freebase_movie_id            0
imdb_id                      0
item_id                  52387
movie_name                   0
combined_release_date        0
movie_countries_final        0
movie_genres_final           0
Box_Office               55185
sa_score                 54903
avgRating                53340
vote_average              9696
vote_count                9705
popularity                   0
dtype: int64

missing item_id is not a problem because that just mean that it comes from the CMU dataset.

In [7]:
# We don't have information about inflation before 1913, so we will apply the 1913 inflation rates to the oldest movies
final_dataset['adjusted_release_date'] = final_dataset['combined_release_date'].apply(inflation_date_constraint)
final_dataset.head()

,freebase_movie_id,imdb_id,item_id,movie_name,combined_release_date,movie_countries_final,movie_genres_final,Box_Office,sa_score,avgRating,vote_average,vote_count,popularity,adjusted_release_date
0,/m/03vyhn,228333,4735.0,Ghosts of Mars,2001-08-24,United States of America,"Thriller, Science Fiction, Horror, Adventure, ...",14010832.0,0.351293,2.35723,5.127,977.0,14.189,2001-08-24
1,/m/08yl5d,245916,NaN,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,United States of America,"Mystery, Biographical film, Drama, Crime Drama",NaN,NaN,NaN,NaN,NaN,0.600,2000-02-16
2,/m/0crgdbh,094806,NaN,Brun bitter,1988-11-17,Norway,"Crime Fiction, Drama",NaN,NaN,NaN,NaN,NaN,0.898,1988-11-17
3,/m/0285_cd,094320,NaN,White Of The Eye,1987-06-19,United Kingdom,"Thriller, Erotic thriller, Psychological thriller",NaN,NaN,NaN,5.742,64.0,8.297,1987-06-19
4,/m/01mrr1,083949,NaN,A Woman in Flames,1983-05-11,Germany,Drama,NaN,NaN,NaN,5.300,13.0,2.801,1983-05-11


In [ ]:
# Adjust the box office revenue with inflation
cpi.update()
# Our column is already in datetime format, but we doublecheck
final_dataset['combined_release_date'] = pd.to_datetime(final_dataset['combined_release_date'], errors='coerce')
# We are going to adjust the inflation to the latest year in the dataset so we have a comparable metric
max_year = final_dataset['adjusted_release_date'].dt.year.max()
# It takes time so we use tqdm to show the progress
tqdm.pandas()
# We compute the inflation adjusted box office revenue and skip the NaN values
final_dataset['inflation_adjusted_box_office_revenue']=final_dataset.progress_apply(
    lambda r: cpi.inflate(r['Box_Office'], r['adjusted_release_date'].year, to=max_year) if pd.notna(r['Box_Office']) else np.nan,
    axis=1
)
# We directly save the dataset to a pickle file because the conversion takes 1 hour
final_dataset.to_pickle('./data/Box_Office_Inflation.pkl')

The final step is to reorder the columns and save the dataset, preparing it for computing our success metric.

In [9]:
# Reorder the columns and sorting the dataset by popularity
data_box_office = pd.read_pickle('./data/Box_Office_Inflation.pkl')
data_box_office.head()
desired_order = ['freebase_movie_id', 'imdb_id', 'item_id', 'movie_name', 'combined_release_date',  'Box_Office', 'inflation_adjusted_box_office_revenue','movie_countries_final', 'movie_genres_final', 'label', 'avgRating', 'vote_average', 'vote_count', 'popularity']
data_box_office = data_box_office.reindex(columns=desired_order)
data_box_office = data_box_office.rename(columns={'label': 'sa_score'})
data_box_office = data_box_office.sort_values(by='popularity', ascending=False)
data_box_office.to_pickle('./data/Data_For_Score.pkl')
data_box_office.head()

,freebase_movie_id,imdb_id,item_id,movie_name,combined_release_date,Box_Office,inflation_adjusted_box_office_revenue,movie_countries_final,movie_genres_final,sa_score,avgRating,vote_average,vote_count,popularity
64821,/m/01n910,087985,NaN,Red Dawn,1984-08-10,4.000000e+07,1.126679e+08,United States of America,"Thriller, Cold War, Action/Adventure, Drama, W...",NaN,NaN,6.272,691.0,300.738
42148,/m/0gmbk1g,630029,NaN,Avatar 2,2022-12-14,2.320250e+09,2.320250e+09,United States of America,"Science Fiction, Action, Fantasy, Adventure",NaN,NaN,7.654,9830.0,241.285
62959,/m/0gdpng,104014,154996.0,All Ladies Do It,1992-02-21,NaN,NaN,Italy,"Softcore Porn, Comedy, World cinema, Adult",0.777778,3.42308,4.800,335.0,232.565
4726,/m/03m9db,283877,27473.0,American Psycho 2,2002-04-22,NaN,NaN,United States of America,"Thriller, Horror, Slasher, Psychological thriller",0.257778,1.98768,4.500,353.0,225.504
58645,/m/03176f,241527,4896.0,Harry Potter and the Philosopher's Stone,2001-11-16,9.747554e+08,1.610768e+09,"United States of America, United Kingdom","Fantasy Adventure, Adventure, Children's/Famil...",0.639775,3.68820,7.916,25379.0,185.482
